In [6]:
import pandas as pd
import keras
import numpy as np

# Import CIFAR 10 dataset
from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator

# Import Necessary CNN Building Blocks
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU

from keras import backend as K
from keras import applications
from keras import callbacks
from keras import optimizers
from keras.utils import np_utils
import random

import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# Requirements:
# !pip install keras
# !pip install tensorflow
# !pip install keras
# !pip install tensorflow_addons

# TODO: data augmentation validation

## Data

In [8]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


## Fitting the Deep Learning Models

### Custom Models

In [9]:
# Normalize input data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Convert class labels to one-hot encoded
# y_train = keras.utils.np_utils.to_categorical(y_train, 10)
# y_test = keras.utils.np_utils.to_categorical(y_test, 10)

In [10]:
tqdm_callback = tfa.callbacks.TQDMProgressBar()

LR=3e-3 # 1e-4
EPOCHS=3  # 200

In [11]:
# # Scheduler of learning rate (decay with epochs)
# def lrScheduler(epoch):
#     return LR * 0.9 ** epoch

### Pretrained Models

#### Residual Network (ResNet)

https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b

In [4]:
LR = 2e-5
EPOCHS=5 # 10

In [27]:
def preprocess_data(X, y):
    """
    a function that trains a convolutional neural network to classify the CIFAR 10 dataset
    
    :param X: X is a numpy.ndarray of shape(m, 32, 32, 3) containing the CIFAR 10 data,
    where m is the number of data points
    :param y: y is a numpy.ndarray of shape(m,) containing the CIFAR 10 data,
    labels for X
    :return: X_p, y_p
        X_p is a numpy.ndarray containing the preprocessed X
        y_p is a numpy.ndarray containing the preprocessed y
    """
    X_p = tf.keras.applications.resnet50.preprocess_input(X)
    y_p = tf.keras.utils.to_categorical(y, 10)
    return X_p, y_p

In [28]:
X_train_resnet50, y_train_resnet50 = preprocess_data(X_train, y_train)
X_test_resnet50, y_test_resnet50 = preprocess_data(X_test, y_test)

input_tensor=tf.keras.Input(shape=X_train_resnet50.shape[1:])

In [29]:
res_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

for layer in res_model.layers[:143]:
    layer.trainable=False
    
resNet = Sequential()
resNet.add(res_model)
resNet.add(tf.keras.layers.Flatten())
resNet.add(tf.keras.layers.Dense(10, activation='softmax'))

In [30]:
resNet.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.RMSprop(learning_rate=LR), 
               metrics=['accuracy'])

In [31]:
resNet.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_4 (Flatten)         (None, 2048)              0         
                                                                 
 dense_10 (Dense)            (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 14,996,490
Non-trainable params: 8,611,712
_________________________________________________________________


In [32]:
resNet.fit(X_train_resnet50,
           y_train_resnet50,
           batch_size=BATCH_SIZE,
           epochs=EPOCHS,
           shuffle=True,
           verbose=1,
           validation_data=(X_test_resnet50, y_test_resnet50)
          )

Epoch 1/10
391/391 [==============================] - 982s 3s/step - loss: 1.9412 - accuracy: 0.4174 - val_loss: 1.4056 - val_accuracy: 0.5417
Epoch 2/10
391/391 [==============================] - 988s 3s/step - loss: 1.0850 - accuracy: 0.6395 - val_loss: 1.2232 - val_accuracy: 0.6122
Epoch 3/10
391/391 [==============================] - 980s 3s/step - loss: 0.7825 - accuracy: 0.7352 - val_loss: 1.1423 - val_accuracy: 0.6361
Epoch 4/10
391/391 [==============================] - 982s 3s/step - loss: 0.5872 - accuracy: 0.8035 - val_loss: 1.1092 - val_accuracy: 0.6509
Epoch 5/10
391/391 [==============================] - 979s 3s/step - loss: 0.4388 - accuracy: 0.8618 - val_loss: 1.1027 - val_accuracy: 0.6570
Epoch 6/10
391/391 [==============================] - 996s 3s/step - loss: 0.3233 - accuracy: 0.9075 - val_loss: 1.0933 - val_accuracy: 0.6647
Epoch 7/10
391/391 [==============================] - 982s 3s/step - loss: 0.2350 - accuracy: 0.9405 - val_loss: 1.1156 - val_accuracy: 0.6650

#### VGG16

In [12]:
EPOCHS=5  # 50
BATCH_SIZE = 32
LR=1e-3
MOMENTUM=0.9

In [13]:
# def preprocess_data(X, y):
#     """
#     a function that trains a convolutional neural network to classify the CIFAR 10 dataset
    
#     :param X: X is a numpy.ndarray of shape(m, 32, 32, 3) containing the CIFAR 10 data,
#     where m is the number of data points
#     :param y: y is a numpy.ndarray of shape(m,) containing the CIFAR 10 data,
#     labels for X
#     :return: X_p, y_p
#         X_p is a numpy.ndarray containing the preprocessed X
#         y_p is a numpy.ndarray containing the preprocessed y
#     """
#     #X_p = tf.keras.applications.vgg16.preprocess_input(X)
#     X_p = X
#     y_p = tf.keras.utils.to_categorical(y, 10)
#     return X_p, y_p

X_train_vgg16 = X_train
y_train_vgg16 = tf.keras.utils.to_categorical(y_train)
X_test_vgg16 = X_test
y_test_vgg16 = tf.keras.utils.to_categorical(y_test)

# nb_train_samples = X_train_vgg16.shape[0]
# nb_validation_samples = X_test_vgg16.shape[0]
# input_tensor=tf.keras.Input(shape=X_train_vgg16.shape[1:])

In [19]:
# base_vgg = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
# # Extract the last layer from third block of vgg16 model
# last = base_vgg.get_layer('block3_pool').output
# # Add classification layers on top of it
# x = Flatten()(last)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# pred = Dense(10, activation='sigmoid')(x)

# vggNet = Model(base_vgg.input, pred)

# PREVIOUSLY COMMENTED OUT
# # set the base model's layers to non-trainable
# # uncomment next two lines if you don't want to
# # train the base model
# # for layer in base_vgg.layers:
# #     layer.trainable = False

#Defining the VGG Convolutional Neural Net
vgg_base = tf.keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_shape=INPUT_SHAPE)

#Adding the final layers to the above base models where the actual classification is done in the dense layers

vgg = Sequential()
vgg.add(base_model) #Adds the base model (in this case vgg19 to model)
vgg.add(Flatten()) #Since the output before the flatten layer is a matrix we have to use this function to get a vector of the form nX1 to feed it into the fully connected layers

#Adding the Dense layers along with activation and batch normalization
# model.add(Dense(1024,activation=('relu'),input_dim=512))
# model.add(Dense(512,activation=('relu'))) 


# model.add(Dense(256,activation=('relu'))) 
# model.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
# model.add(Dense(128,activation=('relu')))
# model.add(Dropout(.2))

vgg.add(Dense(10,activation=('softmax'))) #This is the classification layer
vgg.summary()

#Defining the parameters
# batch_size= 100
# epochs=20
# learn_rate=.001

#sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
#adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,719,818
Trainable params: 14,719,818
Non-trainable params: 0
_________________________________________________________________


In [20]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
vgg.compile(loss='binary_crossentropy',
                   optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM),
              metrics=['accuracy'])

In [17]:
# vggNet.summary()

In [21]:
#Training the model
vgg_history = vgg.fit(X_train_vgg16,
    y_train_vgg16, batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
                      validation_data=(X_test_vgg16, y_test_vgg16),validation_steps=X_test.shape[0]//BATCH_SIZE,
                      verbose=1)

Epoch 1/5
1562/1562 [==============================] - 958s 613ms/step - loss: 0.1774 - accuracy: 0.6234 - val_loss: 0.1416 - val_accuracy: 0.7101
Epoch 2/5
 292/1562 [====>.........................] - ETA: 12:09 - loss: 0.1257 - accuracy: 0.7487

KeyboardInterrupt: 

In [ ]:
# fine-tune the model
# vggNet.fit(
#     # train_generator,
#     X_train_vgg16,
#     y_train_vgg16,
#     steps_per_epoch=nb_train_samples//BATCH_SIZE,
#     epochs=EPOCHS,
#     # validation_data=validation_generator,
#     validation_data=(X_test_vgg16, y_test_vgg16),
#     validation_steps=nb_validation_samples)

## Models Evaluation

#### Residual Network (ResNet)

In [53]:
# TODO
resNet.evaluate(X_test_resnet50, y_test_resnet50)

313/313 [==============================] - 21s 69ms/step - loss: 1.1916 - accuracy: 0.6716


[1.1915991306304932, 0.6715999841690063]

In [54]:
y_pred_test_resnet = resNet.predict(X_test_resnet50)
y_pred_test_classes_resnet = np.argmax(y_pred_test_resnet, axis=-1)

In [55]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_resnet))

,0,1,2,3,4,5,6,7,8,9
0,718,32,45,11,23,9,7,17,93,45
1,27,736,5,14,9,20,9,11,47,122
2,62,12,582,54,101,53,83,33,12,8
3,17,32,63,492,64,170,85,41,13,23
4,14,3,67,56,631,46,84,80,13,6
5,7,14,45,160,53,611,45,52,4,9
6,10,12,60,55,66,47,730,7,6,7
7,23,16,24,50,63,66,15,708,9,26
8,72,52,19,10,13,8,0,6,772,48
9,45,108,1,15,9,12,8,23,43,736


In [56]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_resnet))

              precision    recall  f1-score   support

           0       0.72      0.72      0.72      1000
           1       0.72      0.74      0.73      1000
           2       0.64      0.58      0.61      1000
           3       0.54      0.49      0.51      1000
           4       0.61      0.63      0.62      1000
           5       0.59      0.61      0.60      1000
           6       0.68      0.73      0.71      1000
           7       0.72      0.71      0.72      1000
           8       0.76      0.77      0.77      1000
           9       0.71      0.74      0.73      1000

    accuracy                           0.67     10000
   macro avg       0.67      0.67      0.67     10000
weighted avg       0.67      0.67      0.67     10000



#### VGG16

In [21]:
# TODO
vggNet.evaluate(X_test_vgg16, y_test_vgg16)

313/313 [==============================] - 54s 172ms/step - loss: nan - accuracy: 0.1000


[nan, 0.10000000149011612]

In [22]:
y_pred_test_vgg16 = vggNet.predict(X_test_vgg16)
y_pred_test_classes_vgg16 = np.argmax(y_pred_test_vgg16, axis=-1)

In [28]:
pd.DataFrame(confusion_matrix(y_actual_test_classes, y_pred_test_classes_vgg16))

,0,1,2,3,4,5,6,7,8,9
0,1000,0,0,0,0,0,0,0,0,0
1,1000,0,0,0,0,0,0,0,0,0
2,1000,0,0,0,0,0,0,0,0,0
3,1000,0,0,0,0,0,0,0,0,0
4,1000,0,0,0,0,0,0,0,0,0
5,1000,0,0,0,0,0,0,0,0,0
6,1000,0,0,0,0,0,0,0,0,0
7,1000,0,0,0,0,0,0,0,0,0
8,1000,0,0,0,0,0,0,0,0,0
9,1000,0,0,0,0,0,0,0,0,0


In [29]:
print(classification_report(y_actual_test_classes, y_pred_test_classes_vgg16))

              precision    recall  f1-score   support

           0       0.10      1.00      0.18      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.00      0.00      0.00      1000
           5       0.00      0.00      0.00      1000
           6       0.00      0.00      0.00      1000
           7       0.00      0.00      0.00      1000
           8       0.00      0.00      0.00      1000
           9       0.00      0.00      0.00      1000

    accuracy                           0.10     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.10      0.02     10000



C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ejowik001\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
